In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install ultralytics
!pip install supervision

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 750.4/750.4 kB 10.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-man

In [ ]:
from ultralytics import YOLO
import supervision as sv
from PIL import Image
import cv2
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
MODEL = "yolov8x.pt"
model = YOLO(MODEL)
model.fuse()

100%|██████████| 131M/131M [00:00<00:00, 284MB/s]


YOLOv8x summary (fused): 268 layers, 68200608 parameters, 0 gradients, 257.8 GFLOPs


In [ ]:
from IPython.display import HTML
from base64 import b64encode

In [ ]:
# def play_video(video_path, width=200):
#   mp4 = open(video_path,'rb').read()
#   data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
#   return HTML(f"""<video width={width} controls><source src="{data_url}" type="video/mp4"></video>""")

In [ ]:
path = '/content/drive/MyDrive/behavior2/dataset'
archivos = os.listdir(f'{path}/videos_cond_erratica')
# Verificar si el archivo CSV existe
csv_path = f'{path}/extracted_error.csv'

# Crear un DataFrame vacío si el archivo no existe
if not os.path.exists(csv_path):
    df = pd.DataFrame(columns=['video'])
    df.to_csv(csv_path, index=False)
else:
    df = pd.read_csv(csv_path, index_col=0)

pending_proc = [a for a in archivos if a not in df['video'].tolist()]

In [ ]:
len(pending_proc)

33

In [ ]:
# class_ids of interest - car, motorcycle, bus and truck
selected_classes = [2, 3, 5, 7]

for video in pending_proc:
    video_path = f'{path}/videos_cond_erratica/{video}'
    name_video = video.split('.')[0]
    print(video_path, '-', name_video)
    # Informacion del video
    video_info = sv.VideoInfo.from_video_path(video_path=video_path)
    height, width = video_info.height, video_info.width
    # Frames del video
    frames_generator = sv.get_video_frames_generator(source_path=video_path)#se genera un iterador para acceder a cada frame del video
    frames_list = [frame for frame in frames_generator]
    # Seguimiento
    tracker = sv.ByteTrack()
    # Extraccion imagenes de trayectoria del video
    imgs = {}
    end_img = frames_list[-1]
    #u = len(frames_list)-1
    print('longitud de frames:',len(frames_list))
    for i, frame in enumerate(frames_list):
        print('forma imagen:',frame.shape)
        result = model(frame)[0]
        detections = sv.Detections.from_ultralytics(result)
        detections = tracker.update_with_detections(detections)
        #list_detection = [d for d in detections]
        for xyxy, mask, confidence, class_id, tracker_id, data in detections:
            #print('mask:',mask)
            if class_id in selected_classes:
                list_id = list(imgs.keys())
                if tracker_id not in list_id:
                    imgs[tracker_id] = {'img':np.zeros((height,width,3), np.uint8),'class':class_id}
                    #imgs[tracker_id] = {'img':end_img.copy(),'class':class_id}


                # Obtener xy a partir de xyxy
                xyxy = xyxy.reshape(1,4)
                xy = np.array(
                            [
                                (xyxy[:, 0] + xyxy[:, 2]) / 2,
                                (xyxy[:, 1] + xyxy[:, 3]) / 2,
                            ]
                        ).transpose()
                # Modifica img
                imgs[tracker_id]['img'] = cv2.polylines(
                            imgs[tracker_id]['img'],
                            [xy.astype(np.int32)],
                            True,
                            color=(255,0,0),
                            thickness=10
                        )
    print('id:',imgs.keys())
    # Guardar imagenes
    for i, item in enumerate(imgs.items()):
        k, v = item
        print('clase:',v['class'],' -   id:',k)
        image = Image.fromarray(v['img'])
        image.save(f'{path}/imgs_error2/{name_video}_{i}.png')
        plt.imshow(image)
        plt.show()
    # Agregar video a videos procesados
    df = pd.concat([df, pd.DataFrame([video], columns=['video'])], axis=0)
    df.to_csv(f'{path}/extracted_error.csv')
    #print(df)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
path = '/content/drive/MyDrive/behavior2/dataset'
archivos = os.listdir(f'{path}/videos_cond_normal')
# Verificar si el archivo CSV existe
csv_path = f'{path}/extracted_normal5.csv'

# Crear un DataFrame vacío si el archivo no existe
if not os.path.exists(csv_path):
    df = pd.DataFrame(columns=['video'])
    df.to_csv(csv_path, index=False)
else:
    df = pd.read_csv(csv_path, index_col=0)

pending_proc5 = [a for a in archivos if a not in df['video'].tolist()]

In [ ]:
len(pending_proc5)

24

In [ ]:
# class_ids of interest - car, motorcycle, bus and truck
selected_classes = [2, 3, 5, 7]

for video in pending_proc5:
    video_path = f'{path}/videos_cond_normal/{video}'
    name_video = video.split('.')[0]
    print(video_path, '-', name_video)
    # Informacion del video
    video_info = sv.VideoInfo.from_video_path(video_path=video_path)
    height, width = video_info.height, video_info.width
    # Frames del video
    frames_generator = sv.get_video_frames_generator(source_path=video_path)
    frames_list = [frame for frame in frames_generator]
    # Seguimiento
    tracker = sv.ByteTrack()
    # Extraccion imagenes de trayectoria del video
    imgs = {}
    end_img = frames_list[-1]
    #u = len(frames_list)-1
    print('longitud de frames:',len(frames_list))
    for i, frame in enumerate(frames_list):
        print('forma imagen:',frame.shape)
        result = model(frame)[0]
        detections = sv.Detections.from_ultralytics(result)
        detections = tracker.update_with_detections(detections)
        #list_detection = [d for d in detections]
        for xyxy, mask, confidence, class_id, tracker_id, data in detections:
            #print('mask:',mask)
            if class_id in selected_classes:
                list_id = list(imgs.keys())
                if tracker_id not in list_id:
                    imgs[tracker_id] = {'img':np.zeros((height,width,3), np.uint8),'class':class_id}
                    #imgs[tracker_id] = {'img':end_img.copy(),'class':class_id}


                # Obtener xy a partir de xyxy
                xyxy = xyxy.reshape(1,4)
                xy = np.array(
                            [
                                (xyxy[:, 0] + xyxy[:, 2]) / 2,
                                (xyxy[:, 1] + xyxy[:, 3]) / 2,
                            ]
                        ).transpose()
                # Modifica img
                imgs[tracker_id]['img'] = cv2.polylines(
                            imgs[tracker_id]['img'],
                            [xy.astype(np.int32)],
                            True,
                            color=(255,0,0),
                            thickness=10
                        )
    print('id:',imgs.keys())
    # Guardar imagenes
    for i, item in enumerate(imgs.items()):
        k, v = item
        print('clase:',v['class'],' -   id:',k)
        image = Image.fromarray(v['img'])
        image.save(f'{path}/imgs_normal2/{name_video}_{i}.png')
        plt.imshow(image)
        plt.show()
    # Agregar video a videos procesados
    df = pd.concat([df, pd.DataFrame([video], columns=['video'])], axis=0)
    df.to_csv(f'{path}/extracted_normal5.csv')
    #print(df)

Output hidden; open in https://colab.research.google.com to view.